# Second Try - Natural Language Processing with Distributed Word Vectors


Using Libaries:
*   pandas  -  .

## Training Word2Vec Model

In [7]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "./data/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "./data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "./data/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [9]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z0-9 ]","", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [11]:
# Load the punkt tokenizer
import nltk
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [8]:
%%capture --no-stdout
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


In [9]:
# Check how many sentences we have in total - should be around 850,000+
print (len(sentences))

print (sentences[0])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'ive', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [14]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


In [11]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "output/300features_40minwords_10context"
model.save(model_name)

2017-06-12 23:00:38,935 : INFO : 'pattern' package not found; tag filters are not available for English
2017-06-12 23:00:38,946 : INFO : collecting all words and their counts
2017-06-12 23:00:38,947 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-12 23:00:39,023 : INFO : PROGRESS: at sentence #10000, processed 218259 words, keeping 21425 word types
2017-06-12 23:00:39,100 : INFO : PROGRESS: at sentence #20000, processed 436619 words, keeping 31870 word types


Training model...


2017-06-12 23:00:39,171 : INFO : PROGRESS: at sentence #30000, processed 648646 words, keeping 39901 word types
2017-06-12 23:00:39,246 : INFO : PROGRESS: at sentence #40000, processed 867354 words, keeping 47249 word types
2017-06-12 23:00:39,317 : INFO : PROGRESS: at sentence #50000, processed 1078866 words, keeping 53578 word types
2017-06-12 23:00:39,389 : INFO : PROGRESS: at sentence #60000, processed 1293167 words, keeping 59116 word types
2017-06-12 23:00:39,462 : INFO : PROGRESS: at sentence #70000, processed 1508931 words, keeping 64375 word types
2017-06-12 23:00:39,535 : INFO : PROGRESS: at sentence #80000, processed 1720977 words, keeping 69268 word types
2017-06-12 23:00:39,608 : INFO : PROGRESS: at sentence #90000, processed 1937118 words, keeping 74408 word types
2017-06-12 23:00:39,681 : INFO : PROGRESS: at sentence #100000, processed 2151486 words, keeping 79023 word types
2017-06-12 23:00:39,754 : INFO : PROGRESS: at sentence #110000, processed 2363706 words, keeping 

2017-06-12 23:00:44,991 : INFO : PROGRESS: at sentence #750000, processed 16201667 words, keeping 283158 word types
2017-06-12 23:00:45,074 : INFO : PROGRESS: at sentence #760000, processed 16413665 words, keeping 285472 word types
2017-06-12 23:00:45,158 : INFO : PROGRESS: at sentence #770000, processed 16632999 words, keeping 288096 word types
2017-06-12 23:00:45,248 : INFO : PROGRESS: at sentence #780000, processed 16855219 words, keeping 290568 word types
2017-06-12 23:00:45,335 : INFO : PROGRESS: at sentence #790000, processed 17074482 words, keeping 293103 word types
2017-06-12 23:00:45,387 : INFO : collected 294492 word types from a corpus of 17193356 raw words and 795538 sentences
2017-06-12 23:00:45,389 : INFO : Loading a fresh vocabulary
2017-06-12 23:00:45,725 : INFO : min_count=40 retains 16783 unique words (5% of original 294492, drops 277709)
2017-06-12 23:00:45,727 : INFO : min_count=40 leaves 16355186 word corpus (95% of original 17193356, drops 838170)
2017-06-12 23:00

2017-06-12 23:01:54,553 : INFO : PROGRESS: at 21.72% examples, 194970 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:01:55,560 : INFO : PROGRESS: at 21.92% examples, 193864 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:01:56,591 : INFO : PROGRESS: at 22.12% examples, 192718 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:01:57,592 : INFO : PROGRESS: at 22.33% examples, 191786 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:01:58,640 : INFO : PROGRESS: at 22.48% examples, 190266 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:01:59,728 : INFO : PROGRESS: at 22.66% examples, 188883 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:02:00,768 : INFO : PROGRESS: at 22.83% examples, 187655 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:02:01,876 : INFO : PROGRESS: at 23.01% examples, 186291 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:02:02,897 : INFO : PROGRESS: at 23.19% examples, 185279 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:02:03,908 : INFO : PROGRESS: at 23.39% examples, 184317 wor

2017-06-12 23:03:18,694 : INFO : PROGRESS: at 37.04% examples, 148236 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:03:19,802 : INFO : PROGRESS: at 37.26% examples, 148039 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:03:20,868 : INFO : PROGRESS: at 37.47% examples, 147837 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:03:21,905 : INFO : PROGRESS: at 37.66% examples, 147623 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:03:22,915 : INFO : PROGRESS: at 37.88% examples, 147527 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:03:23,999 : INFO : PROGRESS: at 38.12% examples, 147455 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:03:25,010 : INFO : PROGRESS: at 38.38% examples, 147495 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:03:26,054 : INFO : PROGRESS: at 38.59% examples, 147327 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:03:27,112 : INFO : PROGRESS: at 38.80% examples, 147147 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:03:28,117 : INFO : PROGRESS: at 38.98% examples, 146888 wor

2017-06-12 23:04:42,558 : INFO : PROGRESS: at 53.38% examples, 137612 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:04:43,584 : INFO : PROGRESS: at 53.54% examples, 137432 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:04:44,635 : INFO : PROGRESS: at 53.73% examples, 137300 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:04:45,644 : INFO : PROGRESS: at 53.91% examples, 137193 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:04:46,687 : INFO : PROGRESS: at 54.08% examples, 137008 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:04:47,747 : INFO : PROGRESS: at 54.19% examples, 136699 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:04:48,759 : INFO : PROGRESS: at 54.32% examples, 136447 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:04:49,813 : INFO : PROGRESS: at 54.43% examples, 136145 words/s, in_qsize 8, out_qsize 1
2017-06-12 23:04:50,826 : INFO : PROGRESS: at 54.57% examples, 135928 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:04:51,919 : INFO : PROGRESS: at 54.77% examples, 135811 wor

2017-06-12 23:06:07,158 : INFO : PROGRESS: at 67.84% examples, 128654 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:06:08,161 : INFO : PROGRESS: at 68.02% examples, 128580 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:06:09,198 : INFO : PROGRESS: at 68.17% examples, 128451 words/s, in_qsize 8, out_qsize 1
2017-06-12 23:06:10,203 : INFO : PROGRESS: at 68.34% examples, 128379 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:06:11,242 : INFO : PROGRESS: at 68.52% examples, 128316 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:06:12,248 : INFO : PROGRESS: at 68.73% examples, 128311 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:06:13,257 : INFO : PROGRESS: at 68.92% examples, 128260 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:06:14,299 : INFO : PROGRESS: at 69.13% examples, 128239 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:06:15,326 : INFO : PROGRESS: at 69.31% examples, 128181 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:06:16,326 : INFO : PROGRESS: at 69.47% examples, 128091 wor

2017-06-12 23:07:30,937 : INFO : PROGRESS: at 82.74% examples, 124426 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:07:31,983 : INFO : PROGRESS: at 82.95% examples, 124418 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:07:33,002 : INFO : PROGRESS: at 83.16% examples, 124419 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:07:34,038 : INFO : PROGRESS: at 83.38% examples, 124415 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:07:35,156 : INFO : PROGRESS: at 83.57% examples, 124367 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:07:36,166 : INFO : PROGRESS: at 83.71% examples, 124266 words/s, in_qsize 6, out_qsize 1
2017-06-12 23:07:37,227 : INFO : PROGRESS: at 83.86% examples, 124151 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:07:38,276 : INFO : PROGRESS: at 83.99% examples, 124040 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:07:39,367 : INFO : PROGRESS: at 84.15% examples, 123934 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:07:40,458 : INFO : PROGRESS: at 84.32% examples, 123864 wor

2017-06-12 23:08:55,039 : INFO : PROGRESS: at 96.26% examples, 119832 words/s, in_qsize 6, out_qsize 0
2017-06-12 23:08:56,159 : INFO : PROGRESS: at 96.45% examples, 119788 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:08:57,184 : INFO : PROGRESS: at 96.62% examples, 119753 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:08:58,197 : INFO : PROGRESS: at 96.77% examples, 119693 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:08:59,285 : INFO : PROGRESS: at 96.97% examples, 119672 words/s, in_qsize 8, out_qsize 0
2017-06-12 23:09:00,356 : INFO : PROGRESS: at 97.19% examples, 119685 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:09:01,443 : INFO : PROGRESS: at 97.41% examples, 119692 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:09:02,452 : INFO : PROGRESS: at 97.59% examples, 119676 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:09:03,477 : INFO : PROGRESS: at 97.77% examples, 119643 words/s, in_qsize 7, out_qsize 0
2017-06-12 23:09:04,504 : INFO : PROGRESS: at 97.95% examples, 119623 wor

In [1]:
# Load Model - Restart Point
from gensim.models import Word2Vec
model = Word2Vec.load("output/300features_40minwords_10context")

In [2]:
# Capability Testing
print( model.doesnt_match("man woman child kitchen".split()))
print( model.doesnt_match("france england germany berlin".split()))
print(model.most_similar("man"))
print(model.most_similar("queen"))
print(model.most_similar("awful"))

kitchen
berlin
[('woman', 0.6389591693878174), ('lad', 0.6013070940971375), ('lady', 0.5927731990814209), ('guy', 0.5496988892555237), ('mans', 0.533806324005127), ('boy', 0.5202711820602417), ('monk', 0.5169619917869568), ('soldier', 0.5161561965942383), ('businessman', 0.5152357816696167), ('men', 0.5058982372283936)]
[('princess', 0.6470646858215332), ('bride', 0.6436013579368591), ('stepmother', 0.6263678073883057), ('mistress', 0.6097767353057861), ('eva', 0.6020620465278625), ('countess', 0.6017340421676636), ('kristel', 0.5886794328689575), ('maid', 0.588269054889679), ('prince', 0.5860768556594849), ('belle', 0.5766582489013672)]
[('terrible', 0.7820098996162415), ('horrible', 0.771754264831543), ('atrocious', 0.7529375553131104), ('abysmal', 0.732496976852417), ('dreadful', 0.7215191125869751), ('horrid', 0.6882925033569336), ('appalling', 0.673163890838623), ('horrendous', 0.6601552367210388), ('amateurish', 0.6210966110229492), ('lousy', 0.6142685413360596)]


In [3]:
print(model.wv.syn0)
print(model.wv.syn0.shape)
print(model["flower"][0:30])

[[ 0.02377217 -0.09519248  0.0370535  ...,  0.05187606  0.06247882
   0.03605469]
 [ 0.1496156  -0.01540146 -0.00839629 ...,  0.0186525   0.07152788
  -0.00883645]
 [ 0.10844795 -0.02085898  0.10671753 ...,  0.13860436  0.07949116
   0.0604801 ]
 ..., 
 [-0.02595298  0.01917267 -0.02439976 ..., -0.02773911 -0.02944981
   0.04465172]
 [-0.15748204 -0.07012095  0.01220413 ..., -0.01490343 -0.00587812
   0.00479036]
 [ 0.04520361  0.00827846 -0.03417625 ..., -0.030897    0.05485437
  -0.0643232 ]]
(16783, 300)
[-0.05394089  0.00957698 -0.11943515  0.03507741 -0.00387638  0.05457654
  0.10613424  0.09164964  0.05820949  0.08303292  0.04231497  0.01093064
 -0.04235341  0.01458073 -0.01135401 -0.03930696  0.01984724 -0.07472595
  0.03242525  0.03334251  0.0411524   0.06540743  0.07628481  0.16957279
 -0.05204667 -0.00064853 -0.01422484  0.01205546  0.01634365  0.01417073]


## Attempt 1: Vector Averaging

In [17]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d;" % (counter, len(reviews)), "", end="") 
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [15]:
%%capture --no-stdout
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print( "Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000 Review 1000 of 25000 Review 2000 of 25000 Review 3000 of 25000 Review 4000 of 25000 Review 5000 of 25000 Review 6000 of 25000 Review 7000 of 25000 Review 8000 of 25000 Review 9000 of 25000 Review 10000 of 25000 Review 11000 of 25000 Review 12000 of 25000 Review 13000 of 25000 Review 14000 of 25000 Review 15000 of 25000 Review 16000 of 25000 Review 17000 of 25000 Review 18000 of 25000 Review 19000 of 25000 Review 20000 of 25000 Review 21000 of 25000 Review 22000 of 25000 Review 23000 of 25000 Review 24000 of 25000 Creating average feature vecs for test reviews
Review 0 of 25000 Review 1000 of 25000 Review 2000 of 25000 Review 3000 of 25000 Review 4000 of 25000 Review 5000 of 25000 Review 6000 of 25000 Review 7000 of 25000 Review 8000 of 25000 Review 9000 of 25000 Review 10000 of 25000 Review 11000 of 25000 Review 12000 of 25000 Review 13000 of 25000 Review 14000 of 25000 Review 15000 of 25000 Review 16000 of 25000 Review 17000 of 25000 Review 18000 of 25000 Review 1900

In [16]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "./output/Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


With this being a kaggle dataset, we know that the result has a 0.83044 accuricy rate.<br/>
It does not work as well as bag-of-words, most likely because of the limited amount of learning data.</br>
![Kaggle Results](./img/kaggle_average_vectors.png "0.83044accuricy rate")


## Attempt 2: Clustering

In [25]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  2134.6562077999115 seconds.


In [26]:
# Create a Word / Index dictionary, mapping each vocabulary word to a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [32]:
# Test the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print (words)


Cluster 0
['control', 'pressure']

Cluster 1
['duris', 'epps']

Cluster 2
['support', 'position', 'staff', 'contract', 'defense', 'safety', 'demand', 'property', 'firm', 'services', 'tax', 'protection', 'blackmail', 'request', 'employment', 'institute', 'income', 'earning', 'instructions', 'payment', 'scholarship', 'salary', 'peer']

Cluster 3
['universal', 'republic', 'largest', 'coproduction']

Cluster 4
['erotic', 'healthy', 'perverse', 'visceral', 'glamour', 'eroticism', 'sensuality', 'eyecandy', 'allure', 'sadism', 'undercurrent', 'titillation', 'heady']

Cluster 5
['villainy', 'karyo']

Cluster 6
['smashing', 'grabbing', 'dumping', 'taker']

Cluster 7
['leonardo', 'dicaprio', 'andie']

Cluster 8
['five', 'six', 'seven', '50', 'eight', 'nine', 'twelve', '35', 'fifty']

Cluster 9
['annoying', 'embarrassing', 'irritating', 'obnoxious', 'unappealing', 'grating', 'overacted']


In [33]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [34]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [36]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "./output/BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


With this being a kaggle dataset, we know that the result has a 0.83952 accuricy rate.<br/>
It does not work as well as bag-of-words, it has improved from the original version.</br>
![Kaggle Results](./img/kaggle_clustering.png "0.83952 accuricy rate")